# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [15]:
import pandas as pd
import numpy as np

In [16]:
transcations = pd.read_csv("../data/bank_transactions.csv")
transcations.isna().sum()

type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [17]:
transcations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   type            1000000 non-null  object 
 1   amount          1000000 non-null  float64
 2   nameOrig        1000000 non-null  object 
 3   oldbalanceOrg   1000000 non-null  float64
 4   newbalanceOrig  1000000 non-null  float64
 5   nameDest        1000000 non-null  object 
 6   oldbalanceDest  1000000 non-null  float64
 7   newbalanceDest  1000000 non-null  float64
 8   isFraud         1000000 non-null  int64  
 9   isFlaggedFraud  1000000 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 76.3+ MB


## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

Ans: The current dataset does not contain any missing values. However, there are three non predictive columns, isFlaggedFraud, nameOrig, and nameDest. These columns are not needed for model training, as we will train a model to predict whether a transaction is fraudulent. Additionally, the name columns are not helpful in detecting fraud in the current context. Therefore, these columns can be removed from the dataset.

In [18]:
transcations = transcations.drop(columns=['isFlaggedFraud', 'nameOrig', 'nameDest'])

## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

In [19]:
transcations.groupby('type')["isFraud"].value_counts()

type      isFraud
CASH_IN   0          219955
CASH_OUT  0          350703
          1             657
DEBIT     0            6417
PAYMENT   0          338573
TRANSFER  0           83055
          1             640
Name: count, dtype: int64

In [20]:
encoded_type = pd.get_dummies(transcations['type'])
encoded_type

,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,False,False,False,True,False
1,False,False,False,True,False
2,True,False,False,False,False
3,False,False,False,False,True
4,False,True,False,False,False
...,...,...,...,...,...
999995,False,False,False,True,False
999996,False,False,False,True,False
999997,False,True,False,False,False
999998,False,True,False,False,False


Answer: It seems like cash out and transfer are the two transcations that mostly likeli involed fraud action. To make this pattern usable I can using a encoding method to translate type column into dataset that can use in machine learning

In [21]:
transcations = pd.concat([transcations, encoded_type], axis=1)

In [22]:
transcations.drop(columns=['type'], inplace=True)

In [23]:
transcations.columns

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud', 'CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT',
       'TRANSFER'],
      dtype='object')

## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Ans: Considering there are significantly more non-fraudulent transactions than fraudulent ones, the dataset is imbalanced. This can results in misleading accuracy scores and biased model predictions. To resovle this issue, strategies include oversampling the minority class or undersampling the majority class. In this case, oversampling is the appropriate approach, since it increases the number of fraudulent transcations without losing valuable data.

## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

Answer Here

In [24]:
# write out newly transformed dataset to your folder
transcations.to_csv("../data/transformed_bank_transcations.csv", index=False)